# 03 - Kubernetes 部署 Ray 集群

> **进阶内容**：本 Notebook 为可选的进阶指南，介绍如何在 Kubernetes 上部署 Ray 集群并连接 Daft。如果你只需要在本地使用 Ray，可以跳过本节。

## 学习目标

- 了解在 Kubernetes 上部署 Ray 的意义
- 掌握 KubeRay Operator 的安装和使用
- 学会部署和管理 Ray 集群
- 了解自动扩缩容配置
- 掌握常见故障排除方法

## 1. 概述

### 为什么在 Kubernetes 上部署 Ray？

| 场景 | 本地 Ray | K8s Ray |
|------|---------|--------|
| 资源规模 | 单机 CPU/内存 | 多节点弹性资源 |
| 可用性 | 进程级 | Pod 自动重启 |
| 扩缩容 | 手动 | 自动扩缩容 |
| 资源隔离 | 无 | 命名空间/资源配额 |
| 运维 | 手动管理 | 声明式管理 |

### 架构概览

```
+------------------------------------------+
|           Kubernetes Cluster             |
|                                          |
|  +------------------------------------+  |
|  |       KubeRay Operator             |  |
|  +----------------+-------------------+  |
|                   |                      |
|                   v                      |
|  +------------------------------------+  |
|  |         RayCluster CRD             |  |
|  |                                    |  |
|  |  +----------+  +-----------+       |  |
|  |  | Head Pod |  | Worker Pod|       |  |
|  |  | - GCS    |  | - Raylet  |       |  |
|  |  | - Driver |  | - Tasks   |       |  |
|  |  | - Dashboard| +-----------+      |  |
|  |  +----------+  +-----------+       |  |
|  |                 | Worker Pod|       |  |
|  |                 | - Raylet  |       |  |
|  |                 | - Tasks   |       |  |
|  |                 +-----------+       |  |
|  +------------------------------------+  |
+------------------------------------------+
```

## 2. 前置要求

在开始之前，请确保已安装以下工具：

| 工具 | 用途 | 安装方式 |
|------|------|----------|
| `kubectl` | Kubernetes 命令行工具 | [官方文档](https://kubernetes.io/docs/tasks/tools/) |
| `helm` | Kubernetes 包管理器 | [官方文档](https://helm.sh/docs/intro/install/) |
| `minikube` 或 `kind` | 本地 K8s 集群 | [minikube](https://minikube.sigs.k8s.io/docs/start/) / [kind](https://kind.sigs.k8s.io/) |

In [ ]:
# 检查工具是否已安装（可选执行）
import shutil

tools = ["kubectl", "helm", "minikube", "kind"]
for tool in tools:
    path = shutil.which(tool)
    status = f"已安装 ({path})" if path else "未安装"
    print(f"{tool}: {status}")

## 3. KubeRay Operator

KubeRay 是 Ray 官方的 Kubernetes Operator，负责管理 Ray 集群的生命周期。

### 安装 KubeRay Operator

```bash
# 添加 KubeRay Helm 仓库
helm repo add kuberay https://ray-project.github.io/kuberay-helm/
helm repo update

# 安装 KubeRay Operator
helm install kuberay-operator kuberay/kuberay-operator \
  --version 1.3.0 \
  --namespace kuberay-system \
  --create-namespace

# 验证安装
kubectl get pods -n kuberay-system
```

预期输出：
```
NAME                                READY   STATUS    RESTARTS   AGE
kuberay-operator-xxxxxxxxxx-xxxxx   1/1     Running   0          30s
```

> 也可以使用项目提供的脚本一键安装：`bash ../scripts/setup_ray_k8s.sh`

## 4. 部署 Ray 集群

### 创建命名空间

```bash
kubectl apply -f ../k8s/namespace.yaml
```

### RayCluster 配置详解

项目提供了 `../k8s/ray-cluster.yaml` 配置文件，关键配置说明：

```yaml
apiVersion: ray.io/v1
kind: RayCluster
metadata:
  name: ray-demo-cluster
  namespace: ray-demo
spec:
  rayVersion: '2.53.0'
  headGroupSpec:
    rayStartParams:
      dashboard-host: '0.0.0.0'    # 允许外部访问 Dashboard
    template:
      spec:
        containers:
        - name: ray-head
          image: rayproject/ray:2.53.0
          resources:
            limits:
              cpu: "2"              # Head 节点 2 CPU
              memory: "4Gi"         # Head 节点 4GB 内存
          ports:
          - containerPort: 6379     # GCS 端口
          - containerPort: 8265     # Dashboard 端口
          - containerPort: 10001    # Client 端口
  workerGroupSpecs:
  - replicas: 2                     # 2 个 Worker
    minReplicas: 1                  # 最少 1 个
    maxReplicas: 4                  # 最多 4 个
    template:
      spec:
        containers:
        - name: ray-worker
          image: rayproject/ray:2.53.0
          resources:
            limits:
              cpu: "2"              # 每个 Worker 2 CPU
              memory: "4Gi"         # 每个 Worker 4GB 内存
```

### 部署集群

```bash
kubectl apply -f ../k8s/ray-cluster.yaml

# 查看集群状态
kubectl get rayclusters -n ray-demo
kubectl get pods -n ray-demo
```

预期输出：
```
NAME               STATUS   AGE
ray-demo-cluster   Ready    60s

NAME                                          READY   STATUS    RESTARTS   AGE
ray-demo-cluster-head-xxxxx                   1/1     Running   0          60s
ray-demo-cluster-worker-default-xxxxx-xxxxx   1/1     Running   0          55s
ray-demo-cluster-worker-default-xxxxx-xxxxx   1/1     Running   0          55s
```

## 5. 连接远程 Ray 集群

### 端口转发

```bash
# 转发 Ray Client 端口
kubectl port-forward -n ray-demo svc/ray-demo-cluster-head-svc 10001:10001 &

# 转发 Dashboard 端口
kubectl port-forward -n ray-demo svc/ray-demo-cluster-head-svc 8265:8265 &
```

### 从 Daft 连接

```python
import daft

# 连接远程 Ray 集群
daft.set_runner_ray("ray://localhost:10001")

# 之后的操作与本地 Ray Runner 完全相同
df = daft.read_parquet("s3://your-bucket/data.parquet")
result = df.where(df["price"] > 100).collect()
```

> **注意**：连接远程集群时，数据路径需要使用集群可访问的路径（如 S3、GCS），而非本地路径。

## 6. Dashboard 监控

### 访问方式

端口转发后，访问 http://localhost:8265 即可打开 Ray Dashboard。

### 关键监控指标

| 页面 | 关键指标 | 说明 |
|------|---------|------|
| **Cluster** | Node 状态 | 各节点 CPU/内存/GPU 使用率 |
| **Jobs** | Job 列表 | 提交的任务状态和耗时 |
| **Actors** | Actor 列表 | 活跃 Actor 的资源占用 |
| **Logs** | Worker 日志 | 各节点的运行日志 |
| **Metrics** | 系统指标 | 任务吞吐量、Object Store 使用率 |

## 7. 自动扩缩容

KubeRay 支持根据负载自动调整 Worker 数量。

### 配置说明

在 `ray-cluster.yaml` 的 `workerGroupSpecs` 中：

```yaml
workerGroupSpecs:
- replicas: 2          # 初始 Worker 数
  minReplicas: 1       # 最小 Worker 数（空闲时缩容到此）
  maxReplicas: 4       # 最大 Worker 数（负载高时扩容到此）
```

### 扩缩容行为

```
Load Low          Load Medium        Load High
+--------+        +--------+         +--------+
| Worker |        | Worker |         | Worker |
+--------+        +--------+         +--------+
                  | Worker |         | Worker |
                  +--------+         +--------+
                                     | Worker |
                                     +--------+
                                     | Worker |
                                     +--------+
minReplicas=1     replicas=2         maxReplicas=4
```

- **扩容**：当 Ray 调度器发现资源不足时，KubeRay 自动创建新 Worker Pod
- **缩容**：当 Worker 空闲超过一定时间后，KubeRay 自动删除多余 Worker Pod

## 8. 故障排除

### 常见问题

| 问题 | 可能原因 | 解决方法 |
|------|---------|----------|
| Pod 处于 Pending 状态 | 资源不足 | `kubectl describe pod <pod>` 查看事件，增加集群资源 |
| Pod 处于 CrashLoopBackOff | 配置错误或镜像问题 | `kubectl logs <pod>` 查看日志 |
| 无法连接 Ray Client | 端口转发未启动 | 检查 `kubectl port-forward` 是否运行 |
| Dashboard 无法访问 | 端口冲突或未转发 | 检查 8265 端口是否被占用 |
| Worker 无法加入集群 | 网络策略限制 | 检查 NetworkPolicy 和 Service 配置 |
| OOMKilled | 内存不足 | 增加 `resources.limits.memory` |

### 常用调试命令

```bash
# 查看 Pod 状态
kubectl get pods -n ray-demo -o wide

# 查看 Pod 详情
kubectl describe pod <pod-name> -n ray-demo

# 查看 Pod 日志
kubectl logs <pod-name> -n ray-demo

# 进入 Pod 调试
kubectl exec -it <pod-name> -n ray-demo -- bash

# 查看 RayCluster 状态
kubectl describe raycluster ray-demo-cluster -n ray-demo

# 查看事件
kubectl get events -n ray-demo --sort-by='.lastTimestamp'
```

## 9. 清理资源

```bash
# 删除 Ray 集群
kubectl delete -f ../k8s/ray-cluster.yaml

# 删除命名空间
kubectl delete -f ../k8s/namespace.yaml

# 卸载 KubeRay Operator
helm uninstall kuberay-operator -n kuberay-system
kubectl delete namespace kuberay-system
```

> 也可以使用项目提供的脚本一键清理：`bash ../scripts/cleanup.sh`

## 总结

| 概念 | 说明 |
|------|------|
| KubeRay Operator | 管理 Ray 集群生命周期的 K8s Operator |
| RayCluster CRD | 声明式定义 Ray 集群的配置 |
| Head Pod | 运行 GCS、Dashboard、Driver 的主节点 |
| Worker Pod | 执行 Tasks 和 Actors 的工作节点 |
| 自动扩缩容 | 根据负载自动调整 Worker 数量 |
| 端口转发 | 通过 kubectl 访问集群内部服务 |

## 练习

1. **本地集群**：使用 minikube 或 kind 创建本地 K8s 集群，部署 Ray 并运行 02 中的数据处理任务。
2. **资源调优**：修改 `ray-cluster.yaml`，调整 Worker 的 CPU 和内存配置，观察对性能的影响。

---

恭喜完成 Demo 2！你已经学会了：
- Ray 的核心概念（Tasks、Actors、Object Store）
- Daft + Ray 分布式数据处理
- 在 Kubernetes 上部署 Ray 集群

下一步：继续学习 Demo 3 —— LanceDB 向量数据库基础。